In [7]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

sys.path.append('..')
from audio_toolbox.metrics import splitTabularPredicting

audio_root_folder = '../archive/Data'
labels_csv = os.path.join(audio_root_folder, 'data.csv')
unslicedData = pd.read_csv(labels_csv, header=0)
slicedCSV = os.path.join(audio_root_folder, 'features.csv')
slicedDataSet = pd.read_csv(slicedCSV, header=0)
label = unslicedData['label']
unslicedData = unslicedData.sample(frac=1).reset_index(drop=True)

In [8]:
#from sklearn.preprocessing import StandardScaler

#scalar = StandardScaler()

In [9]:
#generate the testing data folds
K = 5
testDataFolds = []
length = len(unslicedData)
slice_size = length // K
start_index = 0
for i in range(K):
    end_index = min(start_index + slice_size, length)
    fold = unslicedData.iloc[start_index: end_index]
    start_index = end_index
    testDataFolds.append(fold)

In [10]:
#generate the training data folds
trainData = []
for i in range(K):
    index = testDataFolds[i]['index']
    trainData.append(slicedDataSet[~slicedDataSet['index'].isin(index)])

In [11]:
#model training
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(max_depth = 15, max_features = 5)
#model = LogisticRegression(max_iter=1000, C=1e-3)
#model = SVC(decision_function_shape='ovo')

predictions = []
for i in range(K):
    X_train = trainData[i]
    label = X_train['label']
    X_train.drop(['label','index'], axis=1, inplace=True)
    model.fit(X_train, label)
    trainAccuracy = accuracy_score(model.predict(X_train), label)
    print(f"training accuracy: {trainAccuracy}")
    prediction= splitTabularPredicting(model, testDataFolds[i], slicedDataSet)
    predictions.append(prediction)

C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_34032\140797405.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['label','index'], axis=1, inplace=True)


training accuracy: 0.9987490617963473


C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_34032\140797405.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['label','index'], axis=1, inplace=True)


training accuracy: 0.998623967975982


C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_34032\140797405.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['label','index'], axis=1, inplace=True)


training accuracy: 1.0


C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_34032\140797405.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['label','index'], axis=1, inplace=True)


training accuracy: 0.9986237958213436


C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_34032\140797405.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['label','index'], axis=1, inplace=True)


training accuracy: 0.9986234513828056


In [12]:
#average accuracy
accuracies = []
for i in range(K):
    testLabel = testDataFolds[i]['label']
    prediction = predictions[i]
    accuracy = accuracy_score(testLabel, prediction)
    accuracies.append(accuracy)

mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy: {mean_accuracy}")

Mean Accuracy: 0.7326633165829146
